In [26]:
import requests
import json
import os
import pandas as pd
import pymysql
import sqlalchemy
from datetime import datetime

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from joblib import dump

In [27]:
sql_user = "root"
sql_password = "password"
sql_host = "localhost"
sql_port = "3306"
sql_database = "DST_AIRLINES"

connection_string = f"mysql+pymysql://{sql_user}:{sql_password}@{sql_host}:{sql_port}/{sql_database}"
engine = sqlalchemy.create_engine(connection_string)

flights_df = pd.read_sql_table(table_name="flights", con=engine)
weather_df = pd.read_sql_table(table_name="weather_forecasts", con=engine)

cols_to_drop = [
    'Departure_ScheduledTimeLocal_DateTime',
    # 'Departure_ScheduledTimeUTC_DateTime',
    'Departure_ActualTimeLocal_DateTime',
    'Departure_ActualTimeUTC_DateTime',
    # 'Departure_TimeStatus_Code',
    'Departure_TimeStatus_Definition',
    'Arrival_ScheduledTimeLocal_DateTime',
    # 'Arrival_ScheduledTimeUTC_DateTime',
    'Arrival_ActualTimeLocal_DateTime',
    # 'Arrival_ActualTimeUTC_DateTime',
    'Arrival_EstimatedTimeLocal_DateTime',
    'Arrival_EstimatedTimeUTC_DateTime',
    'Departure_EstimatedTimeLocal_DateTime',
    'Departure_EstimatedTimeUTC_DateTime',
    # 'Flight_DateTime',
    # 'Flight_DateTime_Hour',
    'Departure_Terminal_Name',
    'Departure_Terminal_Gate',
    'Arrival_Terminal_Name',
    'Arrival_Terminal_Gate',
    'ServiceType',
    'Departure_AirportCode',
    # 'Arrival_AirportCode',
    'MarketingCarrier_AirlineID',
    'MarketingCarrier_FlightNumber',
    'OperatingCarrier_AirlineID',
    'OperatingCarrier_FlightNumber',
    'Equipment_AircraftCode',
    'Equipment_AircraftRegistration',
    'FlightStatus_Code',
    # 'Airport_Code',
    # 'Latitude',
    # 'Longitude',
    # Valeurs status = inutiles car nous cherchons à déterminer le retard, chiffré
    'FlightStatus_Definition',
    'Arrival_TimeStatus_Definition',
    'FlightStatus_Definition'
]

flights_df = flights_df.drop(cols_to_drop, axis=1)
flights_df = flights_df.dropna(subset=['Arrival_ActualTimeUTC_DateTime'])


# CSV
flights_df.to_csv("/home/sanou/DST-Airlines/Airflow/dags/flights.csv", index=False)
weather_df.to_csv("/home/sanou/DST-Airlines/Airflow/dags/weather.csv", index=False)

In [28]:
import pandas as pd

# Charger les données du fichier CSV
flights_df = pd.read_csv("/home/sanou/DST-Airlines/Airflow/dags/flights.csv")

# Convertir en format datetime avec fuseau horaire (UTC si les données sont en UTC)
flights_df['Arrival_ScheduledTimeUTC_DateTime'] = pd.to_datetime(flights_df['Arrival_ScheduledTimeUTC_DateTime'], utc=True)
flights_df['Arrival_ActualTimeUTC_DateTime'] = pd.to_datetime(flights_df['Arrival_ActualTimeUTC_DateTime'], utc=True)

# Calculer le délai avant toute modification de format de date
flights_df['Delay_minutes'] = (flights_df['Arrival_ActualTimeUTC_DateTime'] - flights_df['Arrival_ScheduledTimeUTC_DateTime']).dt.total_seconds() / 60

# Convertir ensuite les dates au format souhaité YYYY-mm-ddTHH-MM
flights_df['Arrival_ScheduledTimeUTC_DateTime'] = flights_df['Arrival_ScheduledTimeUTC_DateTime'].dt.strftime('%Y-%m-%dT%H')#-%M')
flights_df['Arrival_ActualTimeUTC_DateTime'] = flights_df['Arrival_ActualTimeUTC_DateTime'].dt.strftime('%Y-%m-%dT%H')#-%M')

# Sauvegarder dans un nouveau fichier CSV
flights_df.to_csv("/home/sanou/DST-Airlines/Airflow/dags/flights2.csv", index=False)

# Afficher les premières lignes
flights_df.head()


,Departure_ScheduledTimeUTC_DateTime,Departure_TimeStatus_Code,Arrival_AirportCode,Arrival_ScheduledTimeUTC_DateTime,Arrival_ActualTimeUTC_DateTime,Arrival_TimeStatus_Code,Delay_minutes
0,2024-08-06T02:45Z,DL,AYT,2024-08-06T06,2024-08-06T08,DL,132.0
1,2024-08-06T02:45Z,DL,AYT,2024-08-06T06,2024-08-06T06,DL,39.0
2,2024-08-06T02:45Z,DL,PMI,2024-08-06T04,2024-08-06T05,DL,25.0
3,2024-08-06T02:50Z,DL,CFU,2024-08-06T05,2024-08-06T07,DL,164.0
4,2024-08-06T02:50Z,DL,SUF,2024-08-06T05,2024-08-06T06,DL,69.0


In [29]:
# Charger les données du fichier CSV
weather_df = pd.read_csv("/home/sanou/DST-Airlines/Airflow/dags/weather.csv")

# Convertir en format datetime et appliquer le fuseau horaire UTC
weather_df['Flight_DateTime'] = pd.to_datetime(weather_df['Flight_DateTime']).dt.tz_localize('UTC')

# Convertir au format souhaité YYYY-mm-ddTHH-MM
weather_df['Flight_DateTime'] = weather_df['Flight_DateTime'].dt.strftime('%Y-%m-%dT%H')#:%MZ')

# Sauvegarder dans un nouveau fichier CSV
weather_df.to_csv("/home/sanou/DST-Airlines/Airflow/dags/weather2.csv", index=False)

# Afficher les premières lignes
weather_df.head()


,Flight_DateTime,Airport_Code,Latitude,Longitude,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation_probability,precipitation,...,temperature_180m,soil_temperature_0cm,soil_temperature_6cm,soil_temperature_18cm,soil_temperature_54cm,soil_moisture_0_to_1cm,soil_moisture_1_to_3cm,soil_moisture_3_to_9cm,soil_moisture_9_to_27cm,soil_moisture_27_to_81cm
0,2024-08-26T06,AYT,36.898701,30.800501,28.918001,47.0,16.477943,30.977890,0.0,0.0,...,28.118000,33.018002,28.418001,29.518002,30.368000,0.107,0.126,0.170,0.227,0.283
1,2024-08-26T05,IBZ,38.872898,1.373120,25.623001,78.0,21.500586,28.178627,0.0,0.0,...,23.373001,25.323000,26.073000,27.823000,27.473001,0.149,0.151,0.161,0.183,0.223
2,2024-08-26T04,PMI,39.551701,2.738810,23.762501,70.0,17.964170,24.368599,0.0,0.0,...,22.762501,23.362501,24.462500,28.462500,28.462500,0.103,0.117,0.150,0.186,0.213
3,2024-08-26T05,CFU,39.601898,19.911699,25.556000,95.0,24.694891,31.835178,3.0,0.0,...,26.056000,26.506001,26.156000,30.106001,29.356001,0.112,0.161,0.204,0.219,0.249
4,2024-08-26T05,KVA,40.913300,24.619200,24.549999,58.0,15.743982,26.022018,0.0,0.0,...,26.600000,24.850000,24.799999,27.950001,27.549999,0.057,0.076,0.110,0.153,0.196


In [39]:
df = pd.merge(flights_df, weather_df,
                left_on=['Arrival_AirportCode', 'Arrival_ScheduledTimeUTC_DateTime'],
                right_on=['Airport_Code', 'Flight_DateTime'],
                how="left")

new_cols_drop = [
    'Departure_ScheduledTimeUTC_DateTime',
    'Departure_TimeStatus_Code',
    'Arrival_AirportCode',
    'Arrival_ScheduledTimeUTC_DateTime',
    'Arrival_ActualTimeUTC_DateTime',
    'Arrival_TimeStatus_Code',
    # 'Delay_minutes',
    'Flight_DateTime',
    # 'Airport_Code',
    'Latitude',
    'Longitude',
    # 'temperature_2m',
    # 'relative_humidity_2m',
    # 'dew_point_2m',
    # 'apparent_temperature',
    # 'precipitation_probability',
    # 'precipitation',
    # 'rain',
    # 'showers',
    # 'snowfall',
    # 'snow_depth',
    # 'weather_code',
    # 'pressure_msl',
    # 'surface_pressure',
    # 'cloud_cover',
    # 'cloud_cover_low',
    # 'cloud_cover_mid',
    # 'cloud_cover_high',
    # 'visibility',
    # 'evapotranspiration',
    # 'et0_fao_evapotranspiration',
    # 'vapour_pressure_deficit',
    # 'wind_speed_10m',
    # 'wind_speed_80m',
    # 'wind_speed_120m',
    # 'wind_speed_180m',
    # 'wind_direction_10m',
    # 'wind_direction_80m',
    # 'wind_direction_120m',
    # 'wind_direction_180m',
    # 'wind_gusts_10m',
    # 'temperature_80m',
    # 'temperature_120m',
    # 'temperature_180m',
    # 'soil_temperature_0cm',
    # 'soil_temperature_6cm',
    # 'soil_temperature_18cm',
    # 'soil_temperature_54cm',
    # 'soil_moisture_0_to_1cm',
    # 'soil_moisture_1_to_3cm',
    # 'soil_moisture_3_to_9cm',
    # 'soil_moisture_9_to_27cm',
    # 'soil_moisture_27_to_81cm'
]

df = df.drop(columns=new_cols_drop, axis=1)
df = df.drop_duplicates(subset=['Delay_minutes', 'Airport_Code', 'temperature_2m'])
df = df.dropna(subset=['Airport_Code'])


# CSV
df.to_csv("/home/sanou/DST-Airlines/Airflow/dags/df2.csv", index=False)

df = pd.get_dummies(df)

features = df.drop(['Delay_minutes'], axis=1)
target = df['Delay_minutes']


In [37]:
# print(features.head())
print(target.head())

0    132.0
1    132.0
2     39.0
3     39.0
4     25.0
Name: Delay_minutes, dtype: float64
